IMPORT ALL REQUIRED LIBRARIES

In [1]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import OrderedDict
from os import listdir
from os.path import isfile, join
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException
from decimal import Decimal
import time
import math
import json
import requests
import threading

CONNECT TO THE BINANCE ACCOUNT

In [2]:
api_key = 'api_key'
api_secret = 'api_secret'
client = Client(api_key, api_secret)

OPEN ALL REQUIRED FILES

In [3]:
#trading_history = pd.DataFrame(columns=['day', 'time', 'symbol long', 'symbol short', 'price long', 'price short', 
                                        #'z-score opened', 'periods been opened', 'price sold', 'price bought', 'z-score closed', 'profit'])

In [4]:
mypath = 'path to the historical data'
mypath2 = 'path to the simulation trading performance on historical data'
USDT_all_pairs_df = pd.read_excel('path to dataframe with all possible pairs in binance that are combined to USDT and are available to short')
trading_history = pd.read_excel('path to the file with trading history')
trading_history = trading_history.drop(['Unnamed: 0'],1)

C:\Users\User\AppData\Local\Temp/ipykernel_8140/1696300722.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  trading_history = trading_history.drop(['Unnamed: 0'],1)


In [5]:
trading_history

,day,time,symbol long,symbol short,price long,price short,z-score opened,periods been opened,price sold,price bought,z-score closed,profit


FIND REQUIRED PRECISION FOR EVERY SYMBOL ON BINANNCE

In [6]:
info = client.futures_exchange_info()
symbols_n_precision = {}
for item in info['symbols']:
    symbols_n_precision[item['symbol']] = item['quantityPrecision']

ALL USED FUNCTIONS IN THE MODEL

In [7]:
def current_margin_account_balance(asset = None):
    if asset == None:
        asset = 'USDT'
    for i in range(len(client.get_margin_account()['userAssets'])):
        if client.get_margin_account()['userAssets'][i]['asset'] == asset:
            account_details = client.get_margin_account()['userAssets'][i]
            return float(account_details['free'])
            break

In [8]:
def set_budget_to_pair(available_balance, batch_to_trade, percantage_to_trade = None):
    
    if percantage_to_trade == None:
        percantage_to_trade = 0.8
    
    budget_per_pair = available_balance*percantage_to_trade/len(batch_to_trade)
    
    return budget_per_pair

In [9]:
def open_long(symbol, quantity):
    long_open_result = client.create_margin_order(
    symbol=symbol,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
    quantity=quantity
    )
    return long_open_result

In [10]:
def close_long(symbol, precision, long_open_result):
    qty=0
    for i in range(len(long_open_result['fills'])):
        qty += (float(long_open_result['fills'][i]['qty']) - float(long_open_result['fills'][i]['commission']))
    amount_to_sell = math.floor(qty * 10**precision)/10.0**precision
    long_close_result = client.create_margin_order(
    symbol=symbol,
    side=SIDE_SELL,
    type=ORDER_TYPE_MARKET,
    quantity=amount_to_sell
    )
    return long_close_result, amount_to_sell

In [11]:
def open_short(symbol, quantity):
    loan_details = client.create_margin_loan(asset = symbol[:-4], symbol=symbol, amount = quantity)
    short_open_result = client.create_margin_order(
    symbol=symbol,
    side=SIDE_SELL,
    type=ORDER_TYPE_MARKET,
    quantity=quantity
    )
    return short_open_result

In [12]:
def close_short(symbol, precision, short_open_result):
    interest = float(client.get_margin_interest_history(asset=symbol[:-4])['rows'][0]['interest'])
    qty=0
    for i in range(len(short_open_result['fills'])):
        qty += float(short_open_result['fills'][i]['qty'])
    qty += interest
    qty = qty/(1-0.001)
    amount_to_buy = math.ceil(qty * 10**precision)/10.0**precision
    short_close_result = client.create_margin_order(
    symbol=symbol,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
    sideEffectType= 'AUTO_REPAY',
    quantity=amount_to_buy
    )
    client.repay_margin_loan(asset = symbol[:-4], symbol=symbol, amount = interest)
    return short_close_result, amount_to_buy

In [13]:
def set_index(historical_data):   
    ind = []
    for i in range(len(historical_data['Unnamed: 0'])):
        ind.append(dt.datetime.strptime(historical_data['Unnamed: 0'].iloc[i][:-4], '%Y-%m-%d %H:%M:%S'))
    historical_data.index = ind
    historical_data = historical_data.drop(['Unnamed: 0'],1)
    return historical_data

In [14]:
def set_pairs(USDT_all_pairs_df_file):
    all_pairs = []
    ticker1 = USDT_all_pairs_df['symbol1']
    ticker2 = USDT_all_pairs_df['symbol2']
    for pair_id in range(len(ticker1)):
        pair = [ticker1[pair_id],ticker2[pair_id]]
        all_pairs.append(pair)
    return all_pairs

In [15]:
def price_mult_volume(historical_data):
    c1 = historical_data.columns[0].split()[0]
    c = 0
    while 1:
        if historical_data.columns[c].split()[0] != c1:
            break
        c += 1
    i = 2
    while 1:
        if i>len(historical_data.columns):
            break
        historical_data.insert(i,historical_data.columns[i-1].split()[0] + ' close*volume',
                  (historical_data[historical_data.columns[i-1].split()[0]+" close"]*historical_data[historical_data.columns[i-1].split()[0]+" volume"]))
        i += c+1 
    return historical_data

In [16]:
def correlation(historical_data, interval = None, period_start = None, period_end = None, variables = None):
    
    used_symbols = []
    for i in range(len(historical_data.columns)):
        used_symbols.append(historical_data.columns[i].split()[0])
    symbols = list(dict.fromkeys(used_symbols))
        
    if period_start == None:
        D = historical_data.index[0].day
        M = historical_data.index[0].month
        Y = historical_data.index[0].year
        period_start = (dt.datetime(Y,M,D)).strftime("%d %b,%Y")
    
    if period_end == None:
        D = historical_data.index[-1].day
        M = historical_data.index[-1].month
        Y = historical_data.index[-1].year
        period_end = (dt.datetime(Y,M,D)).strftime("%d %b,%Y")
    
    if interval == None:
        interval = len(historical_data)
    
    if interval >= len(historical_data):
        interval = len(historical_data)
    
    if variables == None:
        used_variables = []
        for i in range(len(historical_data.columns)):
            used_variables.append(historical_data.columns[i].split()[-1])
        variables = list(dict.fromkeys(used_variables))
       
    historical_correlation = pd.DataFrame(columns=[variables])
    
    for i in variables:
        x = historical_data[[col for col in historical_data.columns if col.endswith(i)]]
        x = x[str(symbols[0])+' '+str(i)].rolling(interval).corr(x[str(symbols[1])+' '+str(i)]).tolist()
        historical_correlation[i] = x                                                                 
    
            
    new_interval = len(historical_correlation)
    
    return historical_correlation, symbols, variables, new_interval     
    

In [17]:
def mean_std_corr(historical_correlation, variables = None, interval = None):
    
    if variables == None:
        variables = historical_correlation.columns
    
    n_columns = []
    for i in range(len(variables)):
        for j in ['mean','std']:
            n_columns.append(variables[i][0]+'_'+j)
        
    if interval == None:
        interval = len(historical_correlation)
    if interval >= len(historical_correlation):
        interval = len(historical_correlation)
        
    historical_correlation_mean_std = pd.DataFrame(columns=[n_columns])
    new_row = []
    for i in range(len(variables)):
        new_row.append(np.mean(historical_correlation[variables[i][0]].iloc[-interval:])[-1])
        new_row.append(np.std(historical_correlation[variables[i][0]].iloc[-interval:])[-1])

    historical_correlation_mean_std.loc[len(historical_correlation_mean_std)] = new_row
    
    return historical_correlation_mean_std

In [18]:
def final_matches(all_files, USDT_all_pairs_df, interval_correlation = None, interval_mean = None):
    
    
    all_symbols_dic = {}
    all_symbols = []
    for t in list(all_files.keys()):
        df = all_files[t]
        used_symbols = []
        for i in range(len(df.columns)):
            used_symbols.append(df.columns[i].split()[0])
        symbols = list(dict.fromkeys(used_symbols))
        all_symbols_dic[t] = symbols
        all_symbols.extend(symbols)
        
    all_pairs = set_pairs(USDT_all_pairs_df_file=USDT_all_pairs_df)
    
    best_matches = pd.DataFrame(columns=['symbol_1','symbol_2','mean_corr','std_corr','vol*pr_1','vol*pr_2'])

    for k in range(len(all_pairs)):
        
        try:

            key_1 = list({i for i in all_symbols_dic if all_pairs[k][0] in all_symbols_dic[i]})
            key_2 = list({i for i in all_symbols_dic if all_pairs[k][1] in all_symbols_dic[i]})

            if key_1[0] == key_2[0]:

                historical_data = all_files[key_1[0]]
                historical_data = historical_data.loc[~historical_data.index.duplicated(keep='first')]

            else:

                df_1 = all_files[key_1[0]]
                df_2 = all_files[key_2[0]]
                df_1 = df_1[[col for col in df_1.columns if col.startswith(all_pairs[k][0])]]
                df_2 = df_2[[col for col in df_2.columns if col.startswith(all_pairs[k][1])]]
                df_1 = df_1.loc[~df_1.index.duplicated(keep='first')]
                df_2 = df_2.loc[~df_2.index.duplicated(keep='first')]
                historical_data = pd.concat([df_1, df_2], axis=1)

            pair = all_pairs[k]

            historical_data = historical_data[[col for col in historical_data.columns if (col.startswith(all_pairs[k][0]))|(col.startswith(all_pairs[k][1]))]]

            if interval_correlation == None:
                interval_correlation = len(historical_data)


            historical_correlation, symbols, variables, interval = correlation(historical_data, interval = interval_correlation, variables = ['close'])

            if interval_mean == None:
                interval_mean = len(historical_correlation)-1

            historical_correlation_mean_std = mean_std_corr(historical_correlation, interval=interval_mean)

            mean_std_variables = list(historical_correlation_mean_std.keys())
            mean_std_mean = historical_correlation_mean_std['close_mean']
            mean_std_std = historical_correlation_mean_std['close_std']
            mean_std_symbol_1 = np.mean(historical_data[str(all_pairs[k][0])+' close*volume'])
            mean_std_symbol_2 = np.mean(historical_data[str(all_pairs[k][1])+' close*volume'])

            new_row = {'symbol_1':str(all_pairs[k][0]), 
                       'symbol_2':str(all_pairs[k][1]),
                       'mean_corr':mean_std_mean.iloc[0][0],
                       'std_corr':mean_std_std.iloc[0][0],
                       'vol*pr_1':mean_std_symbol_1,
                       'vol*pr_2': mean_std_symbol_2}
            best_matches = best_matches.append(new_row, ignore_index=True)
            print('Completed ',str(len(best_matches.index)),' out of '+str(len(all_pairs)) ,end="\r")
        
        except:
            continue
            
    return best_matches

In [19]:
def best_combinations_calc(all_files, file_name):
    best_combinations = pd.DataFrame(columns=['symbol 1', 'symbol 2', 'period', 'z_score', 'boarder', 'historical_mean_result', 'historical_mean_std'])
    for i in file_name:
        try:
            df = all_files[i]
            df = df[df['mean result']>1.05]
            df['best_ratio'] = df['mean result']-(df['std result']*3)
            df = df.sort_values(by='best_ratio', ascending=False)
            best_index = df.idxmax()[0]
            period_mode = df['period'][best_index]
            z_score_mode = df['z-score'][best_index]
            boarder_mode = df['boarder'][best_index]
            symbol_1 = df['symbol_1'][best_index].split()[0]
            symbol_2 = df['symbol_2'][best_index].split()[0]
            hist_mean_result = df['mean result'][best_index]
            hist_mean_std = df['std result'][best_index]
            new_row = [symbol_1, symbol_2, period_mode, z_score_mode, boarder_mode,hist_mean_result,hist_mean_std]
            best_combinations.loc[len(best_combinations)] = new_row
        except:
            continue

        best_pairs = (np.log(best_combinations['historical_mean_result']-1)+(-np.log(best_combinations['historical_mean_std'])))
        best_pairs_index = list(best_pairs.index)
        best_combinations = best_combinations.iloc[best_pairs_index]
        best_combinations = best_combinations.sort_values(by='historical_mean_std', ascending=True)

    return best_combinations

In [20]:
def open_files(mypath):
    mypath = mypath
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    all_files = {}
    for t in range(len(onlyfiles)):
        df = pd.read_csv(mypath+'\\'+onlyfiles[t])
        df = price_mult_volume(df)
        all_files[onlyfiles[t]] = df
    return all_files

In [21]:
def open_files_for_update(mypath):
    mypath = mypath
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    all_files = {}
    for t in range(len(onlyfiles)):
        df = pd.read_csv(mypath+'\\'+onlyfiles[t])
        all_files[onlyfiles[t]] = df
    return all_files

In [22]:
def open_files_2(mypath):
    mypath = mypath
    file_name = []
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    all_files = {}
    for t in range(len(onlyfiles)):
        df = pd.read_excel(mypath+'\\'+onlyfiles[t])
        all_files[onlyfiles[t]] = df
        file_name.append(onlyfiles[t])
    return all_files, file_name

In [23]:
def assign_symbols(all_files):
    variables = []
    symbols_name_dic = {}
    for i in range(len(list(all_files.keys()))):
        col_names = list(all_files[list(all_files.keys())[i]].columns)
        symbols_name = []
        for j in col_names:
            symbols_name.append(j.split()[0])
        symbols_name = list(dict.fromkeys(symbols_name))
        symbols_name_dic[list(all_files.keys())[i]] = symbols_name
    return symbols_name_dic

In [24]:
def update_historical_data_df(api_key, api_secret, symbols, interval = None, period_start = None, period_end = None, variables = None, days_to_update = None):

    client = Client(api_key, api_secret)
    
    
    if interval == None:
        interval = '15m'
        
    if days_to_update == None:
        days_to_update = 7
    
    if period_start == None:
        period_start = (dt.date.today()- dt.timedelta(days=days_to_update)).strftime("%d %b,%Y")
    
    if period_end == None:
        period_end = str(time.time() * 1000)
            
    if variables == None:
        variables = ['open_time','open', 'high', 'low', 'close', 'volume',
                    'close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
    
    
    historical_data = pd.DataFrame()
    

    for i in symbols:

        klines = client.get_historical_klines(i, interval, start_str=period_start, end_str=period_end) 
        if klines == []:
            print("Completed: ", symbols.index(i), ' out of ', len(symbols), end="\r")
            continue
        data = pd.DataFrame(klines) 
        data.columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
        data[['open', 'high', 'low', 'close', 'volume', 'qav','num_trades','taker_base_vol','taker_quote_vol']] = data[['open', 'high', 'low', 'close', 'volume', 'qav','num_trades','taker_base_vol','taker_quote_vol']].astype(float) #Convert all string data to the float type
        data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.close_time]
        data = data[variables] 
        data.columns = i+' '+data.columns 
        historical_data = historical_data.merge(data,how='outer', right_index=True, left_index=True) 
        if (symbols.index(i)%10 == 0)&(symbols.index(i)!=0):
            print("Completed: ", symbols.index(i)+1, ' out of ', len(symbols), end="\r")
        else:
            print("Completed: ", symbols.index(i)+1, ' out of ', len(symbols), end="\r")
            continue
            

    return historical_data

In [25]:
def update_historical_data_to_history(api_key, api_secret, mypath, days_to_update = None):  
    
    if days_to_update == None:
        days_to_update = 30
    all_files = open_files_for_update(mypath)
    for i in list(all_files.keys()):
        all_files[i].index = all_files[i]['Unnamed: 0']
        all_files[i] = all_files[i].drop(['Unnamed: 0'],1)
        all_files[i].index = pd.to_datetime(all_files[i].index)
    symbols_name_dic = assign_symbols(all_files)


    end_date = round(time.time() * 1000)
    for k in range(len(list(symbols_name_dic.keys()))):
        current_historical_df = all_files[list(symbols_name_dic.keys())[k]]
        start_date = (current_historical_df.index[-1].date()-dt.timedelta(days=days_to_update)).strftime("%d %b,%Y")
        historical_data = update_historical_data_df(api_key = api_key, api_secret = api_secret, symbols = symbols_name_dic[list(symbols_name_dic.keys())[k]], interval = None, period_start = start_date, period_end = end_date, variables = ['close', 'volume'])
        updated_historical_data = pd.concat([current_historical_df, historical_data], ignore_index=False, sort=False)
        updated_historical_data = updated_historical_data.loc[~updated_historical_data.index.duplicated(keep='first')]
        all_files[list(symbols_name_dic.keys())[k]] = updated_historical_data
        updated_historical_data.to_csv(mypath+'\\'+list(symbols_name_dic.keys())[k])
    return all_files

In [26]:
def update_historical_data(api_key, api_secret, current_historical_df, all_symbols_for_trade, days_to_upd):    
    
    end_date = round(time.time() * 1000)
    
    start_date = (current_historical_df.index[-1].date()-dt.timedelta(days=days_to_upd)).strftime("%d %b,%Y")
    historical_data = update_historical_data_df(api_key = api_key, api_secret = api_secret, symbols = all_symbols_for_trade, interval = None, period_start = start_date, period_end = end_date, variables = ['close'])
    updated_historical_data = pd.concat([current_historical_df, historical_data], ignore_index=False, sort=False)
    updated_historical_data = updated_historical_data.loc[~updated_historical_data.index.duplicated(keep='first')]
    if current_historical_df.index[-1] == historical_data.index[-1]:
        data_update = 0
    else:
        data_update = 1
    
    
    for r in list(batch_with_data.keys()):
        sym1 = batch_with_data[r]['symbol1']
        sym2 = batch_with_data[r]['symbol2']
        batch_with_data[r]['data'+sym1] = updated_historical_data[sym1+' close']
        batch_with_data[r]['data'+sym2] = updated_historical_data[sym2+' close']
        his_data_1 = batch_with_data[r]['data'+sym1]
        his_data_2 = batch_with_data[r]['data'+sym2]
        batch_with_data[r]['current_correlation'] = np.corrcoef(his_data_1[-2688:], his_data_2[-2688:])[0][-1]
        ratio = his_data_1[-batch_with_data[r]['period']:]/his_data_2[-batch_with_data[r]['period']:]
        mean = np.mean(ratio)
        std = np.std(ratio)
        batch_with_data[r]['current_z_score'] = ((ratio[-1])-mean)/std
        batch_with_data[r]['updated_price'] = data_update
    
    return batch_with_data, updated_historical_data, data_update
    

UPDATE HISTORICAL DATA

In [44]:
all_files_updated = update_historical_data_to_history(api_key = api_key, api_secret = api_secret, mypath = mypath, days_to_update=40)

C:\Users\User\AppData\Local\Temp/ipykernel_17308/2502402584.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  all_files[i] = all_files[i].drop(['Unnamed: 0'],1)


In [46]:
for i in all_files_updated:
    all_files_updated[i] = price_mult_volume(all_files_updated[i])

FIND HISTORICAL ROLLING CORRELATION FOR EVERY PAIR

In [75]:
best_matches = final_matches(all_files = all_files_updated, USDT_all_pairs_df=USDT_all_pairs_df, interval_correlation=2688)

In [76]:
best_matches.to_csv('path to best correlated pairs')
best_matches = pd.read_csv('path to best correlated pairs')

In [87]:
main_pairs = best_matches
main_pairs['best_ratio'] = main_pairs['mean_corr'] - (main_pairs['std_corr']*3)
main_pairs = main_pairs.sort_values(by='best_ratio', ascending=False)
main_pairs = main_pairs.iloc[:5]

In [88]:
main_pairs

,Unnamed: 0,symbol_1,symbol_2,mean_corr,std_corr,vol*pr_1,vol*pr_2,best_ratio
0,7438,HBARUSDT,KDAUSDT,0.887644,0.079810,2.293747e+05,79168.520925,0.807833
1,532,NEOUSDT,ONTUSDT,0.899381,0.106169,3.621053e+05,306141.889759,0.793213
2,4589,ENJUSDT,MANAUSDT,0.885431,0.120358,4.256189e+05,915462.036462,0.765072
3,875,QTUMUSDT,ONTUSDT,0.878989,0.123883,2.686528e+05,306141.889759,0.755106
4,15567,GALUSDT,OPUSDT,0.859733,0.109175,3.695314e+05,654529.210601,0.750558
5,13301,ALICEUSDT,TLMUSDT,0.881145,0.131078,6.168422e+05,563212.071744,0.750067
6,10516,DOTUSDT,KDAUSDT,0.862901,0.119646,2.520527e+06,79168.520925,0.743255
7,13576,TLMUSDT,TVKUSDT,0.865995,0.124360,5.632121e+05,67008.149694,0.741635
8,526,NEOUSDT,QTUMUSDT,0.872211,0.135456,3.621053e+05,268652.751898,0.736755
9,4674,ENJUSDT,RNDRUSDT,0.837610,0.105726,4.256189e+05,68445.474656,0.731885


In [89]:
all_files = open_files_for_update(mypath)
    
all_symbols_dic = {}
all_symbols = []
for t in list(all_files.keys()):
    df = all_files[t]
    used_symbols = []
    for i in range(len(df.columns)):
        used_symbols.append(df.columns[i].split()[0])
    symbols = list(dict.fromkeys(used_symbols))
    all_symbols_dic[t] = symbols
    all_symbols.extend(symbols)

In [90]:
for i in list(all_files.keys()):
    all_files[i].index = all_files[i]['Unnamed: 0']
    all_files[i] = all_files[i].drop(['Unnamed: 0'],1)

C:\Users\User\AppData\Local\Temp/ipykernel_17308/3069292198.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  all_files[i] = all_files[i].drop(['Unnamed: 0'],1)


In [97]:
pair_1 = main_pairs['symbol_1']
pair_2 = main_pairs['symbol_2']
df_pairs = {}
for k in range(len(pair_1)):

    key_1 = list({i for i in all_symbols_dic if pair_1[k] in all_symbols_dic[i]})
    key_2 = list({i for i in all_symbols_dic if pair_2[k] in all_symbols_dic[i]})

    if key_1[0] == key_2[0]:

        historical_data = all_files[key_1[0]]
        historical_data = historical_data[[col for col in historical_data.columns if (col.startswith(pair_1[k]))|(col.startswith(pair_2[k]))]]
        historical_data = historical_data[[col for col in historical_data.columns if col.endswith('close')]]
        df_pairs[k] = historical_data


    else:

        df_1 = all_files[key_1[0]]
        df_2 = all_files[key_2[0]]
        df_1 = df_1.astype('float32')
        df_2 = df_2.astype('float32')
        df_1 = df_1[[col for col in df_1.columns if col.startswith(pair_1[k])]]
        df_2 = df_2[[col for col in df_2.columns if col.startswith(pair_2[k])]]
        historical_data = df_1.join(df_2)
        historical_data = historical_data[[col for col in historical_data.columns if col.endswith('close')]]
        df_pairs[k] = historical_data

ON THE 10 BEST PAIRS BASED ON CORRELATION SIMULATE TRADING ON HISTORICAL DATA WITH DIFFERENT TRADING PARAMETERS

In [ ]:
for i in range(len(df_pairs)):

    trade_period = 2000
    performance = pd.DataFrame(columns=['period','z-score','boarder','symbol_1','symbol_2','mean result','std result',
                                        'mean amount of deals'])

    periods = [4,8,16,32,64,128,256,512]
    z_scores = [1,2,3,4,5]
    boarders = [0,-1,-2,-3]


    symbol1 = df_pairs[i][df_pairs[i].columns[-2]].astype(float)
    symbol2 = df_pairs[i][df_pairs[i].columns[-1]].astype(float)




    for r in periods:
        period = r

        for k in z_scores:
            z_open = k

            for n in boarders:
                boarder = n

                trading = pd.DataFrame(columns=['Time','open or close','Symbol bought','volume bought', 'Price bought','Symbol sold',
                                               'volume sold', 'Price Sold','Z','USDT'])
                percentage = 0.8 
                USDT = 1000
                commision = 0.001
                start_time = time.time()


                USDT = 1000
                trading = pd.DataFrame(columns=['Time','open or close','Symbol bought','volume bought', 'Price bought','Symbol sold',
                                               'volume sold', 'Price Sold','Z','USDT'])



                position_status = 0
                profits_over_time = list([])
                amounts_of_deals = list([])
                for p in np.arange(0,len(df_pairs[i])/trade_period,1):
                    i1 = int(trade_period*p)
                    i2 = np.min([int(trade_period*(p+1)),len(df_pairs[i])-1])
                    symbol_1 = symbol1.iloc[i1:i2]
                    symbol_2 = symbol2.iloc[i1:i2]
                    ratio = symbol_1 / symbol_2

                    for j in range(period, len(ratio)):
                        mean_ratio = ratio[j-period:j].mean()
                        std_ratio = ratio[j-period:j].std()
                        z = (ratio[j] - mean_ratio)/std_ratio

                        if ((z>z_open)&(position_status==0)):
                            position_status = 1

                            volume_buy = ((USDT/len(df_pairs)) * percentage / 2) / symbol_2.iloc[j]
                            volume_sell = ((USDT/len(df_pairs)) * percentage / 2) / symbol_1.iloc[j]
                            USDT = USDT + ((-volume_buy*symbol_2.iloc[j]*(1+commision)) + (volume_sell*symbol_1.iloc[j]*(1-commision)))
                            trading.loc[len(trading.index)] = [df_pairs[i].index[j], 'open', 
                                                               df_pairs[i].columns[-1], volume_buy,
                                                               symbol_2.iloc[j],
                                                               df_pairs[i].columns[-2], volume_sell,
                                                               symbol_1.iloc[j],
                                                               z, USDT]

                        if ((z<-z_open)&(position_status==0)):
                            position_status = -1

                            volume_buy = ((USDT/len(df_pairs)) * percentage / 2) / symbol_1.iloc[j]
                            volume_sell = ((USDT/len(df_pairs)) * percentage / 2) / symbol_2.iloc[j]
                            USDT = USDT + ((-volume_buy*symbol_1.iloc[j]*(1+commision)) + (volume_sell*symbol_2.iloc[j]*(1-commision)))
                            trading.loc[len(trading.index)] = [df_pairs[i].index[j], 'open', 
                                                               df_pairs[i].columns[-2], volume_buy,
                                                               symbol_1.iloc[j],
                                                               df_pairs[i].columns[-1], volume_sell,
                                                               symbol_2.iloc[j],
                                                               z, USDT]

                        if ((z<boarder)&(position_status==1)):
                            position_status = 0

                            volume_buy = trading.loc[len(trading.index)-1]['volume sold']
                            volume_sell = trading.loc[len(trading.index)-1]['volume bought']

                            USDT = USDT + ((-volume_buy*symbol_1.iloc[j]*(1+commision)) + (volume_sell*symbol_2.iloc[j]*(1-commision)))

                            trading.loc[len(trading.index)] = [df_pairs[i].index[j], 'close', 
                                                               df_pairs[i].columns[-2], volume_buy,
                                                               symbol_1.iloc[j],
                                                               df_pairs[i].columns[-1], volume_sell, 
                                                               symbol_2.iloc[j],
                                                               z, USDT]

                        if ((z>-boarder)&(position_status==-1)):
                            position_status = 0

                            volume_buy = trading.loc[len(trading.index)-1]['volume sold']
                            volume_sell = trading.loc[len(trading.index)-1]['volume bought']


                            USDT = USDT + ((-volume_buy*symbol_2.iloc[j]*(1+commision)) + (volume_sell*symbol_1.iloc[j]*(1-commision)))

                            trading.loc[len(trading.index)] = [df_pairs[i].index[j], 'close', 
                                                               df_pairs[i].columns[-1], volume_buy,
                                                               symbol_2.iloc[j],
                                                               df_pairs[i].columns[-2], volume_sell,
                                                               symbol_1.iloc[j],
                                                             z, USDT]       
                    try:
                        profit_over_time = trading['USDT'].iloc[-1]/trading['USDT'].iloc[0]
                        amount_of_deals = int(len(trading)/2)
                        profits_over_time.append(profit_over_time)
                        amounts_of_deals.append(amount_of_deals)
                    except:
                        pass
                try:
                    profits_over_time_mean = np.array(profits_over_time).mean()
                    profits_over_time_std = np.array(profits_over_time).std()
                    amounts_of_deals_mean = np.array(amounts_of_deals).mean()
                    sym_1 = trading['Symbol bought'].iloc[-1]
                    sym_2 = trading['Symbol sold'].iloc[-1]
                    new_row = [r,k,n,sym_1,sym_2,profits_over_time_mean,profits_over_time_std,amounts_of_deals_mean]
                    performance.loc[len(performance.index)] = new_row
                except:
                    pass
                print('Completed ', str(int(len(performance))),' out of ',str(len(periods)*len(z_scores)*len(boarders)),
                      ' with time of ',"--- %s seconds ---" % (time.time() - start_time),end='\r')
    performance.to_excel('path to simulation result on historical data'+str(df_pairs[i].columns[0].split()[0])+'_'+str(df_pairs[i].columns[1].split()[0])+'.xlsx')

In [36]:
all_files, file_name = open_files_2(mypath2)

BASED ON SIMULATION ON HISTORICAL DATA, SELECT BEST PAIRS WITH THE MOST PROFITABLE PARAMETERS

In [37]:
best_combinations = best_combinations_calc(all_files, file_name)
best_combinations['best_ratio'] = best_combinations['historical_mean_result']-(best_combinations['historical_mean_std']*3)
best_combinations = best_combinations.sort_values(by='best_ratio', ascending = False)
best_combinations = best_combinations[:4]
best_combinations

,symbol 1,symbol 2,period,z_score,boarder,historical_mean_result,historical_mean_std,best_ratio
2,ENJUSDT,MANAUSDT,512,3,-2,1.060943,0.016743,1.010713
6,TVKUSDT,TLMUSDT,512,1,-3,1.072108,0.021211,1.008474
0,ALICEUSDT,TLMUSDT,512,4,0,1.052163,0.019210,0.994534
4,NEOUSDT,ONTUSDT,128,3,0,1.051656,0.024734,0.977454


In [243]:
new_batch = pd.DataFrame(columns=['pair ID', 'symbol 1', 'symbol 2', 'period', 'z_score', 'boarder', 'current correlation', 'current z-score', 'available for trading'])
for i in range(len(best_combinations)):
    new_row = []
    new_row.append(USDT_all_pairs_df[((USDT_all_pairs_df['symbol1'] == best_combinations.iloc[i]['symbol 1'])&(USDT_all_pairs_df['symbol2'] == best_combinations.iloc[i]['symbol 2']))|
                 ((USDT_all_pairs_df['symbol2'] == best_combinations.iloc[i]['symbol 1'])&(USDT_all_pairs_df['symbol1'] == best_combinations.iloc[i]['symbol 2']))]['pair ID'].iloc[-1])
    new_row.append(best_combinations.iloc[i]['symbol 1'])
    new_row.append(best_combinations.iloc[i]['symbol 2'])
    new_row.append(best_combinations.iloc[i]['period'])
    new_row.append(best_combinations.iloc[i]['z_score'])
    new_row.append(best_combinations.iloc[i]['boarder'])
    new_row.append(0)
    new_row.append(0)
    new_row.append(0)
    new_batch.loc[len(new_batch.index)] = new_row

CREATE A BATCH WITH BEST PAIRS AND MOST APPROPRIATE PARAMETERS TO USE THEM IN TRADING

In [244]:
new_batch

,pair ID,symbol 1,symbol 2,period,z_score,boarder,current correlation,current z-score,available for trading
0,4673,ENJUSDT,MANAUSDT,512,3,-2,0,0,0
1,13698,ALICEUSDT,TLMUSDT,512,4,0,0,0,0
2,13981,TVKUSDT,TLMUSDT,512,1,-3,0,0,0
3,541,NEOUSDT,ONTUSDT,128,3,0,0,0,0


In [245]:
new_batch.to_excel('path to best selected pairs to trade')

In [ ]:
best_combinations = pd.read_excel('path to best selected pairs to trade')
best_combinations['updated_price'] = 0
best_combinations = best_combinations.drop(['Unnamed: 0'],1)

In [ ]:
all_files = open_files_for_update(mypath)

In [ ]:
for i in list(all_files.keys()):
    all_files[i].index = all_files[i]['Unnamed: 0']
    all_files[i] = all_files[i].drop(['Unnamed: 0'],1)
    all_files[i].index = pd.to_datetime(all_files[i].index)

In [228]:
batch_with_data = {}
all_symbols_for_trade = []
for i in range(len(best_combinations)):
    batch_with_data[best_combinations.iloc[i]['pair ID']] = {}
    batch_with_data[best_combinations.iloc[i]['pair ID']]['pair ID'] = best_combinations.iloc[i]['pair ID']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['symbol1'] = best_combinations.iloc[i]['symbol 1']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['symbol2'] = best_combinations.iloc[i]['symbol 2']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['period'] = best_combinations.iloc[i]['period']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['z_score'] = best_combinations.iloc[i]['z_score']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['boarder'] = best_combinations.iloc[i]['boarder']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['current_correlation'] = best_combinations.iloc[i]['current correlation']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['current_z_score'] = best_combinations.iloc[i]['current z-score']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['available_for_trading'] = best_combinations.iloc[i]['available for trading']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['updated_price'] = best_combinations.iloc[i]['updated_price']
    symbol1 = best_combinations.iloc[i]['symbol 1']
    symbol2 = best_combinations.iloc[i]['symbol 2']
    key1 = list({i for i in all_files if symbol1+' close' in all_files[i]})[-1]
    key2 = list({i for i in all_files if symbol2+' close' in all_files[i]})[-1]
    batch_with_data[best_combinations.iloc[i]['pair ID']]['data'+str(best_combinations.iloc[i]['symbol 1'])] = all_files[key1][symbol1+' close']
    batch_with_data[best_combinations.iloc[i]['pair ID']]['data'+str(best_combinations.iloc[i]['symbol 2'])] = all_files[key2][symbol2+' close']

In [229]:
all_symbols_for_trade = []
for i in range(len(best_combinations)):
    for j in ['symbol 1', 'symbol 2']:
        all_symbols_for_trade.append(best_combinations.iloc[i][j])
all_symbols_for_trade = list(dict.fromkeys(all_symbols_for_trade))

df_with_data = pd.DataFrame()

for j in range(len(all_symbols_for_trade)):
    key = list({i for i in all_files if all_symbols_for_trade[j]+' close' in all_files[i]})[-1]
    df = all_files[key][all_symbols_for_trade[j]+' close']
    df_with_data = df_with_data.merge(df,how='outer', right_index=True, left_index=True)
    df_with_data = df_with_data.loc[~df_with_data.index.duplicated(keep='first')]

In [230]:
df_with_data

,TLMUSDT close,ALICEUSDT close,GALUSDT close,MBOXUSDT close
Unnamed: 0,,,,
2020-12-17 22:44:59.000,NaN,NaN,NaN,NaN
2020-12-17 22:59:59.000,NaN,NaN,NaN,NaN
2020-12-17 23:14:59.000,NaN,NaN,NaN,NaN
2020-12-17 23:29:59.000,NaN,NaN,NaN,NaN
2020-12-17 23:44:59.000,NaN,NaN,NaN,NaN
...,...,...,...,...
2022-09-29 13:29:59.999,0.02181,1.701,2.596,0.627
2022-09-29 13:44:59.999,0.02177,1.703,2.599,0.626
2022-09-29 13:59:59.999,0.02175,1.705,2.599,0.627


In [ ]:
all_files = update_historical_data_to_history(api_key = api_key, api_secret = api_secret, mypath = mypath, days_to_update = 30)

START TRADING MODEL

In [ ]:
available_balance = current_margin_account_balance()
budget_per_pair = set_budget_to_pair(available_balance, batch_with_data)
key = "https://api.binance.com/api/v3/ticker/price?symbol="
trading_cache = {}


batch_to_trade, updated_historical_data, data_update = update_historical_data(api_key = api_key, api_secret = api_secret, current_historical_df = df_with_data,
                                          all_symbols_for_trade = all_symbols_for_trade, days_to_upd = 5)
df_with_data = updated_historical_data
print(batch_to_trade, end='\r')
print('<------------------------>')



while True:
    
    try:
    

        api_key = 'api_key'
        api_secret = 'api_secret'
        client = Client(api_key, api_secret)

        batch_to_trade, updated_historical_data, data_update = update_historical_data(api_key = api_key, api_secret = api_secret, current_historical_df = df_with_data,
                                              all_symbols_for_trade = all_symbols_for_trade, days_to_upd = 1)
        df_with_data = updated_historical_data
        print('<------------------------>')
        if data_update == 1:

            print(batch_to_trade, end='\r')

            for pair in batch_to_trade:

                if ((batch_to_trade[pair]['available_for_trading'] == 0)&(batch_to_trade[pair]['updated_price'] == 1)):

                    if batch_to_trade[pair]['current_z_score'] > batch_to_trade[pair]['z_score']:

                        trading_cache[pair] = {}

                        pair_id = batch_to_trade[pair]['pair ID']

                        trading_cache[pair]['new_deal_to_trading_history'] = []

                        trading_cache[pair]['symbol1'] = batch_to_trade[pair]['symbol1']
                        trading_cache[pair]['symbol2'] = batch_to_trade[pair]['symbol2']
                        trading_cache[pair]['price1_open'] = float(requests.get(key+(trading_cache[pair]['symbol1'])).json()['price'])
                        trading_cache[pair]['price2_open'] = float(requests.get(key+(trading_cache[pair]['symbol2'])).json()['price'])
                        trading_cache[pair]['order_amount1'] = budget_per_pair/2/(trading_cache[pair]['price1_open'])
                        trading_cache[pair]['order_amount2'] = budget_per_pair/2/(trading_cache[pair]['price2_open'])
                        trading_cache[pair]['precision1'] = symbols_n_precision[trading_cache[pair]['symbol1']]
                        trading_cache[pair]['precision2'] = symbols_n_precision[trading_cache[pair]['symbol2']]
                        trading_cache[pair]['precise_order_amount1'] = "{:0.0{}f}".format(trading_cache[pair]['order_amount1'], trading_cache[pair]['precision1'])              
                        trading_cache[pair]['precise_order_amount2'] = "{:0.0{}f}".format(trading_cache[pair]['order_amount2'], trading_cache[pair]['precision2'])

                        trading_cache[pair]['date1'] = dt.datetime.today()
                        
                        trading_cache[pair]['long_open_result'] = open_long(trading_cache[pair]['symbol2'], trading_cache[pair]['precise_order_amount2'])
                        
                        trading_cache[pair]['short_open_result'] = open_short(trading_cache[pair]['symbol1'], trading_cache[pair]['precise_order_amount1'])
                        
                        print('Symbol '+str(trading_cache[pair]['symbol2'])+' was bought, and symbol '+str(trading_cache[pair]['symbol1'])+' was sold')

                        trading_cache[pair]['new_deal_to_trading_history'].append([dt.date.today().strftime("%Y/%m/%d"),
                                                                           dt.datetime.now().strftime('%H:%M:%S'),
                                                                           trading_cache[pair]['symbol2'],
                                                                           trading_cache[pair]['symbol1'],
                                                                           trading_cache[pair]['long_open_result']['fills'][0]['price'],
                                                                           trading_cache[pair]['short_open_result']['fills'][0]['price'],
                                                                           batch_to_trade[pair]['current_z_score']]
                                                                           )

                        batch_to_trade[pair]['available_for_trading'] = 1

                    elif batch_to_trade[pair]['current_z_score'] < -(batch_to_trade[pair]['z_score']):

                        trading_cache[pair] = {}

                        pair_id = batch_to_trade[pair]['pair ID']

                        trading_cache[pair]['new_deal_to_trading_history'] = []

                        trading_cache[pair]['symbol1'] = batch_to_trade[pair]['symbol1']
                        trading_cache[pair]['symbol2'] = batch_to_trade[pair]['symbol2']
                        trading_cache[pair]['price1_open'] = float(requests.get(key+(trading_cache[pair]['symbol1'])).json()['price'])
                        trading_cache[pair]['price2_open'] = float(requests.get(key+(trading_cache[pair]['symbol2'])).json()['price'])
                        trading_cache[pair]['order_amount1'] = budget_per_pair/2/(trading_cache[pair]['price1_open'])
                        trading_cache[pair]['order_amount2'] = budget_per_pair/2/(trading_cache[pair]['price2_open'])
                        trading_cache[pair]['precision1'] = symbols_n_precision[trading_cache[pair]['symbol1']]
                        trading_cache[pair]['precision2'] = symbols_n_precision[trading_cache[pair]['symbol2']]
                        trading_cache[pair]['precise_order_amount1'] = "{:0.0{}f}".format(trading_cache[pair]['order_amount1'], trading_cache[pair]['precision1'])              
                        trading_cache[pair]['precise_order_amount2'] = "{:0.0{}f}".format(trading_cache[pair]['order_amount2'], trading_cache[pair]['precision2'])

                        trading_cache[pair]['date1'] = dt.datetime.today()
                        
                        trading_cache[pair]['long_open_result'] = open_long(trading_cache[pair]['symbol1'], trading_cache[pair]['precise_order_amount1'])
                        
                        trading_cache[pair]['short_open_result'] = open_short(trading_cache[pair]['symbol2'], trading_cache[pair]['precise_order_amount2'])

                        print('Symbol '+str(trading_cache[pair]['symbol1'])+' was bought, and symbol '+str(trading_cache[pair]['symbol2'])+' was sold')

                        trading_cache[pair]['new_deal_to_trading_history'].append([dt.date.today().strftime("%Y/%m/%d"),
                                                                           dt.datetime.now().strftime('%H:%M:%S'),
                                                                           trading_cache[pair]['symbol1'],
                                                                           trading_cache[pair]['symbol2'],
                                                                           trading_cache[pair]['long_open_result']['fills'][0]['price'],
                                                                           trading_cache[pair]['short_open_result']['fills'][0]['price'],
                                                                           batch_to_trade[pair]['current_z_score']]
                                                                           )

                        batch_to_trade[pair]['available_for_trading'] = -1

                    else:
                        continue

                elif ((batch_to_trade[pair]['available_for_trading'] == 1)&(batch_to_trade[pair]['updated_price'] == 1)):

                    if batch_to_trade[pair]['current_z_score'] < batch_to_trade[pair]['boarder']:
                        
                        trading_cache[pair]['long_close_result'], trading_cache[pair]['amount_to_sell'] = close_long(trading_cache[pair]['symbol2'], trading_cache[pair]['precision2'], trading_cache[pair]['long_open_result'])
                        
                        try:
                            trading_cache[pair]['short_close_result'], trading_cache[pair]['amount_to_buy'] = close_short(trading_cache[pair]['symbol1'], trading_cache[pair]['precision1'], trading_cache[pair]['short_open_result'])
                        except:
                            continue

                        trading_cache[pair]['date2'] = dt.datetime.today()
                        trading_cache[pair]['diff'] = int(np.round((trading_cache[pair]['date2']-trading_cache[pair]['date1']).seconds/60/15,0))

                        trading_cache[pair]['long_open_cost'] = -(trading_cache[pair]['precise_order_amount2']*(trading_cache[pair]['long_open_result']['fills'][0]['price']))
                        trading_cache[pair]['short_open_cost'] = (trading_cache[pair]['precise_order_amount1']*(trading_cache[pair]['short_open_result']['fills'][0]['price']))

                        trading_cache[pair]['long_close_cost'] = (trading_cache[pair]['amount_to_sell']*(trading_cache[pair]['long_close_result']['fills'][0]['price']))
                        trading_cache[pair]['short_close_cost'] = -(trading_cache[pair]['amount_to_buy']*(trading_cache[pair]['short_close_result'][0]['fills'][0]['price']))

                        trading_cache[pair]['profit'] = trading_cache[pair]['long_open_cost']+trading_cache[pair]['short_open_cost']+trading_cache[pair]['long_close_cost']+trading_cache[pair]['short_close_cost']

                        trading_cache[pair]['new_deal_to_trading_history'].append([trading_cache[pair]['diff'],
                                                                           trading_cache[pair]['long_close_result']['fills'][0]['price'],
                                                                           trading_cache[pair]['short_close_result'][0]['fills'][0]['price'],
                                                                           batch_to_trade[pair]['current_z_score'],
                                                                           trading_cache[pair]['profit']]
                                                                           )
                        trading_cache[pair]['new_deal_to_trading_history'] = trading_cache[pair]['new_deal_to_trading_history'][0]+trading_cache[pair]['new_deal_to_trading_history'][1]
                        trading_history.loc[len(trading_history.index)] = trading_cache[pair]['new_deal_to_trading_history']
                        
                        print('Symbol '+str(trading_cache[pair]['symbol1'])+' was REbought, and symbol '+str(trading_cache[pair]['symbol2'])+' was REsold')
                        print(trading_cache[pair]['profit'])
                        batch_to_trade[pair]['available_for_trading'] = 0

                    else:
                        continue

                elif ((batch_to_trade[pair]['available_for_trading'] == -1)&(batch_to_trade[pair]['updated_price'] == 1)):

                    if batch_to_trade[pair]['current_z_score'] > -(batch_to_trade[pair]['boarder']):
                        
                        trading_cache[pair]['long_close_result'], trading_cache[pair]['amount_to_sell'] = close_long(trading_cache[pair]['symbol1'], trading_cache[pair]['precision1'], trading_cache[pair]['long_open_result'])
                        
                        try:
                            trading_cache[pair]['short_close_result'], trading_cache[pair]['amount_to_buy'] = close_short(trading_cache[pair]['symbol2'], trading_cache[pair]['precision2'], trading_cache[pair]['short_open_result'])
                        except:
                            continue

                        trading_cache[pair]['date2'] = dt.datetime.today()
                        trading_cache[pair]['diff'] = int(np.round((trading_cache[pair]['date2']-trading_cache[pair]['date1']).seconds/60/15,0))

                        trading_cache[pair]['long_open_cost'] = -(trading_cache[pair]['precise_order_amount1']*(trading_cache[pair]['long_open_result']['fills'][0]['price']))
                        trading_cache[pair]['short_open_cost'] = (trading_cache[pair]['precise_order_amount2']*(trading_cache[pair]['short_open_result']['fills'][0]['price']))

                        trading_cache[pair]['long_close_cost'] = (trading_cache[pair]['amount_to_sell']*(trading_cache[pair]['long_close_result']['fills'][0]['price']))
                        trading_cache[pair]['short_close_cost'] = -(trading_cache[pair]['amount_to_buy']*(trading_cache[pair]['short_close_result'][0]['fills'][0]['price']))

                        trading_cache[pair]['profit'] = trading_cache[pair]['long_open_cost']+trading_cache[pair]['short_open_cost']+trading_cache[pair]['long_close_cost']+trading_cache[pair]['short_close_cost']




                        trading_cache[pair]['new_deal_to_trading_history'].append([trading_cache[pair]['diff'],
                                                                           trading_cache[pair]['long_open_result']['fills'][0]['price'],
                                                                           trading_cache[pair]['short_close_result'][0]['fills'][0]['price'],
                                                                           batch_to_trade[pair]['current_z_score'],
                                                                           trading_cache[pair]['profit']]
                                                                           )
                        trading_cache[pair]['new_deal_to_trading_history'] = trading_cache[pair]['new_deal_to_trading_history'][0]+trading_cache[pair]['new_deal_to_trading_history'][1]
                        trading_history.loc[len(trading_history.index)] = trading_cache[pair]['new_deal_to_trading_history']
                        
                        print('Symbol '+str(trading_cache[pair]['symbol2'])+' was REbought, and symbol '+str(trading_cache[pair]['symbol1'])+' was REsold')
                        print(trading_cache[pair]['profit'])
                        batch_to_trade[pair]['available_for_trading'] = 0

                    else:
                        continue

                else:
                    continue




            x = 1
            for k in range(830):
                x+=1
                time.sleep(1)
        else:
            time.sleep(5)
            
    except:
        
        time.sleep(5)
        continue




{13698: {'pair ID': 13698, 'symbol1': 'TLMUSDT', 'symbol2': 'ALICEUSDT', 'period': 32, 'z_score': 1, 'boarder': 0, 'current_correlation': 0.6979121234393486, 'current_z_score': -1.8739851580289224, 'available_for_trading': 0, 'updated_price': 1, 'dataTLMUSDT': 2020-12-17 22:44:59.000        NaN
2020-12-17 22:59:59.000        NaN
2020-12-17 23:14:59.000        NaN
2020-12-17 23:29:59.000        NaN
2020-12-17 23:44:59.000        NaN
                            ...   
2022-10-03 15:14:59.999    0.02492
2022-10-03 15:29:59.999    0.02490
2022-10-03 15:44:59.999    0.02476
2022-10-03 15:59:59.999    0.02489
2022-10-03 16:14:59.999    0.02491
Name: TLMUSDT close, Length: 66139, dtype: float64, 'dataALICEUSDT': 2020-12-17 22:44:59.000      NaN
2020-12-17 22:59:59.000      NaN
2020-12-17 23:14:59.000      NaN
2020-12-17 23:29:59.000      NaN
2020-12-17 23:44:59.000      NaN
                           ...  
2022-10-03 15:14:59.999    1.689
2022-10-03 15:29:59.999    1.689
2022-10-03 15:44:59.9

<------------------------>th: 66142, dtype: float64}}
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
{13698: {'pair ID': 13698, 'symbol1': 'TLMUSDT', 'symbol2': 'ALICEUSDT', 'period': 32, 'z_score': 1, 'boarder': 0, 'current_correlation': 0.6941778964266121, 'current_z_score': -1.0839559527686127, 'available_for_trading': -1, 'updated_price': 1, 'dataTLMUSDT': 2020-12-17 22:44:59.000        NaN
2020-12-17 22:59:59.000        NaN
2020-12-17 23:14:59.000        NaN
2020-12-17 23:29:59.000        NaN
2020-12-17 23:44:59.000        NaN
                            ...   
2022-10-03 16:14:59.999    0.02491
2022-10-03 16:29:59.999    0.02489
2022-10-03 16:44:59.999    0.02483
2022-10-03 16:59:59.999    0.02488
2022-10-03 17:14:59.999    0.02490
Name: TLMUSDT close, Length: 66143, dtype: float64, 'dataALICEUSDT': 2020-12-17 22:44:59.000      NaN
2020-12-17 22:59:59.000      NaN
20

<------------------------>th: 66146, dtype: float64}}
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
{13698: {'pair ID': 13698, 'symbol1': 'TLMUSDT', 'symbol2': 'ALICEUSDT', 'period': 32, 'z_score': 1, 'boarder': 0, 'current_correlation': 0.6902079894389104, 'current_z_score': -1.2055261359534644, 'available_for_trading': -1, 'updated_price': 1, 'dataTLMUSDT': 2020-12-17 22:44:59.000        NaN
2020-12-17 22:59:59.000        NaN
2020-12-17 23:14:59.000        NaN
2020-12-17 23:29:59.000        NaN
2020-12-17 23:44:59.000        NaN
                            ...   
2022-10-03 17:14:59.999    0.02490
2022-10-03 17:29:59.999    0.02501
2022-10-03 17:44:59.999    0.02512
2022-10-03 17:59:59.999    0.02507
2022-10-03 18:14:59.999    0.02496
Name: TLMUSDT close, Length: 66147, dtype: float64, 'dataALICEUSDT': 2020-12-17 22:44:59.000      NaN
2020-12-17 22:59:59.000      NaN
2020-12-17 23:14:59.000      

<------------------------>th: 66150, dtype: float64}}
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
{13698: {'pair ID': 13698, 'symbol1': 'TLMUSDT', 'symbol2': 'ALICEUSDT', 'period': 32, 'z_score': 1, 'boarder': 0, 'current_correlation': 0.6868350551692944, 'current_z_score': -1.3082549701295, 'available_for_trading': -1, 'updated_price': 1, 'dataTLMUSDT': 2020-12-17 22:44:59.000        NaN
2020-12-17 22:59:59.000        NaN
2020-12-17 23:14:59.000        NaN
2020-12-17 23:29:59.000        NaN
2020-12-17 23:44:59.000        NaN
                            ...   
2022-10-03 18:14:59.999    0.02496
2022-10-03 18:29:59.999    0.02480
2022-10-03 18:44:59.999    0.02481
2022-10-03 18:59:59.999    0.02459
2022-10-03 19:14:59.999    0.02471
Name: TLMUSDT close, Length: 66151, dtype: float64, 'dataALICEUSDT': 2020-12-17 22:44:59.000      NaN
2020-12-17 

<------------------------>th: 66154, dtype: float64}}
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
{13698: {'pair ID': 13698, 'symbol1': 'TLMUSDT', 'symbol2': 'ALICEUSDT', 'period': 32, 'z_score': 1, 'boarder': 0, 'current_correlation': 0.6836992135448937, 'current_z_score': -1.5564685668076892, 'available_for_trading': -1, 'updated_price': 1, 'dataTLMUSDT': 2020-12-17 22:44:59.000        NaN
2020-12-17 22:59:59.000        NaN
2020-12-17 23:14:59.000        NaN
2020-12-17 23:29:59.000        NaN
2020-12-17 23:44:59.000        NaN
                            ...   
2022-10-03 19:14:59.999    0.02471
2022-10-03 19:29:59.999    0.02476
2022-10-03 19:44:59.999    0.02451
2022-10-03 19:59:59.999    0.02458
2022-10-03 20:14:59.999    0.02460
Name: TLMUSDT close, Length: 66155, dtype: float64, 'dataALICEUSDT': 2020-12-17 22:44:59.000      NaN
2020-12-17 22:59:59.000      NaN
20

<------------------------>th: 66158, dtype: float64}}
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
{13698: {'pair ID': 13698, 'symbol1': 'TLMUSDT', 'symbol2': 'ALICEUSDT', 'period': 32, 'z_score': 1, 'boarder': 0, 'current_correlation': 0.680674991548999, 'current_z_score': -1.9699495465573171, 'available_for_trading': -1, 'updated_price': 1, 'dataTLMUSDT': 2020-12-17 22:44:59.000        NaN
2020-12-17 22:59:59.000        NaN
2020-12-17 23:14:59.000        NaN
2020-12-17 23:29:59.000        NaN
2020-12-17 23:44:59.000        NaN
                            ...   
2022-10-03 20:14:59.999    0.02460
2022-10-03 20:29:59.999    0.02478
2022-10-03 20:44:59.999    0.02474
2022-10-03 20:59:59.999    0.02459
2022-10-03 21:14:59.999    0.02461
Name: TLMUSDT close, Length: 66159, dtype: float64, 'dataALICEUSDT': 2020-12-17 22:44:59.000      NaN
2020-12-17 22:59:59.000      NaN
202

<------------------------>th: 66162, dtype: float64}}
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
<------------------------>
{13698: {'pair ID': 13698, 'symbol1': 'TLMUSDT', 'symbol2': 'ALICEUSDT', 'period': 32, 'z_score': 1, 'boarder': 0, 'current_correlation': 0.678184561636681, 'current_z_score': -2.334771639990335, 'available_for_trading': -1, 'updated_price': 1, 'dataTLMUSDT': 2020-12-17 22:44:59.000        NaN
2020-12-17 22:59:59.000        NaN
2020-12-17 23:14:59.000        NaN
2020-12-17 23:29:59.000        NaN
2020-12-17 23:44:59.000        NaN
                            ...   
2022-10-03 21:14:59.999    0.02461
2022-10-03 21:29:59.999    0.02473
2022-10-03 21:44:59.999    0.02466
2022-10-03 21:59:59.999    0.02456
2022-10-03 22:14:59.999    0.02440
Name: TLMUSDT close, Length: 66163, dtype: float64, 'dataALICEUSDT': 2020-12-17 22:44:59.000      NaN
2020-12-17 22:59:59.000      NaN
2020

In [ ]:
trading_history.to_excel('path to trading history')